<a href="https://colab.research.google.com/github/fabriciopluz/imersao_Agentes_IA_Alura/blob/main/Imers%C3%A3o_Agentes_Alura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imersão de criação de Agentes de IA - Alura


#Aula 01 - Construindo o Agente de IA com Gemini.

In [ ]:
#Instalção das bibliotecas para tratamento dos eventos de Agentes de IA
!pip install -q --upgrade langchain-google-genai google-generativeai

In [ ]:
#Importação das bibliotecas para o ambiente do google colab.
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [ ]:
#Criação da conexão com o modelo de inteligência a ser usado.
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperatura = 0.0,
    api_key = GOOGLE_API_KEY

)

In [ ]:
#testando o uso do objeto llm

resp_test = llm.invoke("Quem é você e quais benefícios você traz para sociedade")
print(resp_test.content)

Olá! Sou um modelo de linguagem grande, treinado pelo Google. Isso significa que eu sou um programa de computador avançado, projetado para entender e gerar texto em linguagem humana. Não tenho um corpo físico, emoções, consciência ou experiências pessoais como um ser humano. Minha "existência" se dá no processamento de dados e algoritmos.

**Em resumo, quem eu sou:**

*   **Uma Inteligência Artificial:** Não sou um ser vivo, mas sim um sistema computacional.
*   **Um Modelo de Linguagem:** Minha principal função é processar e gerar texto, entender contextos, responder perguntas, traduzir, resumir e criar conteúdo.
*   **Treinado com Vastos Dados:** Fui exposto a uma quantidade imensa de textos e códigos da internet para aprender padrões, gramática, fatos e diferentes estilos de comunicação.
*   **Uma Ferramenta:** Sirvo como um assistente, uma fonte de informação e um gerador de ideias.

---

**Quais benefícios trago para a sociedade?**

Minha existência e a de outras IAs semelhantes t

In [ ]:
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."
)

In [ ]:
#Importando blibliotecas para criação das respostas.
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class TriagemOutput(BaseModel):
  decisao: Literal["AUTO_RESOLVER", "PEDIR_INFO", "ABRIR_CHAMADO"]
  urgencia: Literal["BAIXA", "MEDIA", "ALTA"]
  campos_faltantes: List[str]

In [ ]:
#Criando uma llm para triagem, o que vai ser usado no agente.
llm_triagem = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperatura = 0.0,
    api_key = GOOGLE_API_KEY

)

In [ ]:
#Estruturando as mensagens de retorno, quando é uma mensagem de sistema e quando é uma mensagem de usuário.

from langchain_core.messages import SystemMessage, HumanMessage

triagem_cahin = llm_triagem.with_structured_output(TriagemOutput)

def Triagem(mensagem: str) -> Dict:
  saída: TriagemOutput = triagem_cahin.invoke([SystemMessage(content = TRIAGEM_PROMPT),
                                              HumanMessage(content = mensagem)])

  return saída.model_dump()

In [ ]:
#Criando Listas de perguntas.
testes = ["Posso reembolsar a Internet",
          "Posso ter mais 5 dias de trabalho remoto, como faço",
          "Posso reembolsar os pagamentos efetuados em treinamentos da Alura?",
          "Quantas capivaras tem no rio Pinheiros?"]

In [ ]:
#Criando um laço for para passar as mensagem para o Agente.
for test_messagem in testes:
  print(f"-> Pergunta: {test_messagem}\n -> Resposta: {Triagem(test_messagem)}\n")

-> Pergunta: Posso reembolsar a Internet
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

-> Pergunta: Posso ter mais 5 dias de trabalho remoto, como faço
 -> Resposta: {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': []}

-> Pergunta: Posso reembolsar os pagamentos efetuados em treinamentos da Alura?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

-> Pergunta: Quantas capivaras tem no rio Pinheiros?
 -> Resposta: {'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': ['relação com políticas internas']}



# Aula 02 - Construindo a base de conhecimento com RAG

In [ ]:
#Instalando as bibliotecas
!pip install -q --upgrade langchain_community faiss-cpu langchain-text-splitters pymupdf

In [ ]:
#Criando repositório dos documentos em pdf para salvar em uma lista.
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader

docs = []

for n_doc in Path("/content/").glob("*.pdf"):
  try:
      loader = PyMuPDFLoader(str(n_doc))
      docs.extend(loader.load())
      print(f"Carregado arquivo com sucesso{n_doc.name}")
  except Exception as e:
      print(f"Erro ao carregar arquivo {n_doc.name}: {e}")

print(f"Total de arquivos carregados: {len(docs)}")

Carregado arquivo com sucessohome_office.pdf
Carregado arquivo com sucessoreembolso.pdf
Carregado arquivo com sucessoseguranca.pdf
Total de arquivos carregados: 3


In [ ]:
#Importando as blibiotecas para quebrar os arquivos(textos) partes(tamanho) menores

from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size = 300, chunk_overlap = 30)

chunks = splitter.split_documents(docs)

In [ ]:
#Imprimindo as partes separadas )chunks de cada conteúdo de cada pdf.
for chunk in chunks:
    print(chunk)
    print("n\-----------------------------------")

page_content='Políticas de Home Office 
1.​ A empresa adota modelo híbrido: mínimo de 2 dias presenciais por semana, salvo 
exceções aprovadas pelo gestor e RH. 
2.​ Equipamentos: a empresa fornece monitor e periféricos. O colaborador é 
responsável por zelar pela conservação.' metadata={'producer': 'Skia/PDF m141 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': '/content/home_office.pdf', 'file_path': '/content/home_office.pdf', 'total_pages': 1, 'format': 'PDF 1.4', 'title': 'Documento sem título', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}
n\-----------------------------------
page_content='3.​ Segurança: é obrigatório o uso de VPN e bloqueio de tela. Documentos 
confidenciais não devem ser impressos fora do escritório. 
4.​ Ergonomia: recomendamos cadeira adequada e suporte de monitor. O RH pode 
avaliar solicitação de apoio ergonômico.' metadata={'producer': 'Skia/PDF m141 Google Docs 

<>:4: SyntaxWarning: invalid escape sequence '\-'
<>:4: SyntaxWarning: invalid escape sequence '\-'
/tmp/ipython-input-1728570922.py:4: SyntaxWarning: invalid escape sequence '\-'
  print("n\-----------------------------------")


In [ ]:
#Importando as blibliotecas para trabalhar com enbaddings

from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001",
    google_api_key = GOOGLE_API_KEY


)

In [ ]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(chunks, embeddings)

retriever = vectorstore.as_retriever(search_type="similarity_score_threshold",
                                     search_kwargs={"score_threshold":0.3, "k": 4})

In [ ]:
#Importando bibliotecas para criação do prompt do Agente

from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt_rag = ChatPromptTemplate.from_messages([
    ("system",
     "Você é um Assistente de Políticas Internas (RH/IT) da empresa Carraro Desenvolvimento. "
     "Responda SOMENTE com base no contexto fornecido. "
     "Se não houver base suficiente, responda apenas 'Não sei'."),

    ("human", "Pergunta: {input}\n\nContexto:\n{context}")
])

document_chain = create_stuff_documents_chain(llm_triagem, prompt_rag)

In [ ]:
#Criando a função de pergunta

def perguntar_politica_RAG (pergunta: str) -> Dict:
  docs_relacionados = retriever.invoke(pergunta)

  if not docs_relacionados:
    return {"answer": "Não sei",
           "Citações":[],
           "contexto_encontrado":False}

  answer = document_chain.invoke({"input":pergunta,
                                   "context": docs_relacionados})
  txt = (answer or "").strip()

  if txt.rstrip(".!?") == "Não sei":
      return{"answer": "Não sei",
        "Citações":[],
        "contexto_encontrado":False}

  return{"answer": txt,
           "Citações":formatar_citacoes(docs_relacionados,pergunta),
           "contexto_encontrado":True}

In [ ]:
#Criando os testes para o Agente
testes = ["Posso reembolsar a Internet",
          "Posso ter mais 5 dias de trabalho remoto, como faço",
          "Posso reembolsar os pagamentos efetuados em treinamentos da Alura?",
          "Quantas capivaras tem no rio Pinheiros?"]

In [ ]:
# Formatadores
import re, pathlib

def _clean_text(s: str) -> str:
    return re.sub(r"\s+", " ", s or "").strip()

def extrair_trecho(texto: str, query: str, janela: int = 240) -> str:
    txt = _clean_text(texto)
    termos = [t.lower() for t in re.findall(r"\w+", query or "") if len(t) >= 4]
    pos = -1
    for t in termos:
        pos = txt.lower().find(t)
        if pos != -1: break
    if pos == -1: pos = 0
    ini, fim = max(0, pos - janela//2), min(len(txt), pos + janela//2)
    return txt[ini:fim]

def formatar_citacoes(docs_rel: List, query: str) -> List[Dict]:
    cites, seen = [], set()
    for d in docs_rel:
        src = pathlib.Path(d.metadata.get("source","")).name
        page = int(d.metadata.get("page", 0)) + 1
        key = (src, page)
        if key in seen:
            continue
        seen.add(key)
        cites.append({"documento": src, "pagina": page, "trecho": extrair_trecho(d.page_content, query)})
    return cites[:3]

In [ ]:
#criando um laço for para o teste.

for msg_teste in testes:
    resposta = perguntar_politica_RAG(msg_teste)
    print(f"PERGUNTA: {msg_teste}")
    print(f"RESPOSTA: {resposta['answer']}")
    if resposta['contexto_encontrado']:
      print("CITAÇÕES:")
    for c in resposta["Citações"]:
        print(f"- Documento: {c['documento']}, Página: {c['pagina']}")
        print(f"  Trecho: {c['trecho']}")


PERGUNTA: Posso reembolsar a Internet
RESPOSTA: Sim, a internet para home office é reembolsável via subsídio mensal de até R$ 100, mediante nota fiscal nominal.
CITAÇÕES:
- Documento: Política de Reembolsos (Viagens e Despesas).pdf, Página: 1
  Trecho: lsáveis.​ 3.​ Transporte: táxi/app são permitidos quando não houver alternativa viável. Comprovantes obrigatórios.​ 4.​ Internet para home office: reembolsável via subsídio mensal de até R$ 100, conforme política de Home Office.​
- Documento: Políticas de Home Office.pdf, Página: 1
  Trecho: 5.​ Conectividade: há subsídio mensal de internet domiciliar para quem trabalha em home office: até R$ 100/mês, mediante nota fiscal nominal.​ 6.​ Solicitação de
PERGUNTA: Posso ter mais 5 dias de trabalho remoto, como faço
RESPOSTA: Sim, é possível solicitar uma exceção para ter 5 dias de trabalho remoto.

Para fazer isso, você deve formalizar a solicitação via chamado ao RH com a justificativa do seu gestor.
CITAÇÕES:
- Documento: Políticas de Home